<a href="https://colab.research.google.com/github/yoojinlee-hub/Applications_for_various_situations/blob/main/Window/BOM_auto_python/ReadPDFinPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install library

In [88]:
from google.colab import files
import pdfplumber
import csv
import re
from collections import Counter

# 로컬 파일 선택 창이 열립니다.
uploaded = files.upload()

# 업로드한 PDF 파일의 이름을 확인합니다.
file_names = list(uploaded.keys())
if len(file_names) == 0:
    print("파일이 업로드되지 않았습니다.")
else:
    pdf_file_name = file_names[0]  # 첫 번째 업로드된 파일 이름을 사용

    # PDF 파일을 텍스트로 추출합니다.
    def extract_text_from_pdf(pdf_file_name):
        text = ""
        with pdfplumber.open(pdf_file_name) as pdf:
            for page in pdf.pages:
                text += page.extract_text()
        return text

    # PDF 파일로부터 텍스트 추출
    pdf_text = extract_text_from_pdf(pdf_file_name)

    # 추출한 단어를 텍스트 파일로 저장
    txt_file_name = 'filtered_text_test.txt'  # 저장할 텍스트 파일 이름
    with open(txt_file_name, 'w', encoding='utf-8') as txt_file:
        txt_file.write(pdf_text)

    # 정규표현식 패턴을 적용하여 특정 조건에 맞는 단어 추출
    def process_word(match):
        word = match.group(0)
        # 정규표현식을 사용하여 3번 이상 반복되는 문자 또는 숫자를 하나로 처리
        word = re.sub(r'(.)\1{2,}', r'\1', word)
        return word

    word_pattern = re.compile(r'\b(?![A-Za-z]\b)(?![0-9]+\b)(?![a-zA-Z_/.\-]+\b)\w+\b')

    words = word_pattern.findall(pdf_text)
    processed_words = [process_word(match) for match in word_pattern.finditer(pdf_text)]
    word_counter = Counter(processed_words)

    # CSV 파일로 저장합니다.
    csv_file_name = 'word_count.csv'
    with open(csv_file_name, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['단어', '빈도'])
        for word, count in word_counter.most_common():
            csv_writer.writerow([word, count])

    print(f'"{csv_file_name}" 파일이 생성되었습니다.')


Saving PION_V1_0_20230927.pdf to PION_V1_0_20230927 (1).pdf
"word_count.csv" 파일이 생성되었습니다.


추출본 입력받고 비교하기

In [89]:
!pip install pandas openpyxl

In [90]:
import pandas as pd

In [91]:
# CSV 파일을 DataFrame으로 읽어옵니다.
df_csv = pd.read_csv(csv_file_name)

In [92]:
# 로컬 파일 선택 창이 열립니다.
xlsx_uploaded = files.upload()

# 업로드한 PDF 파일의 이름을 확인합니다.
xlsx_file_names = list(xlsx_uploaded.keys())
if len(xlsx_file_names) == 0:
    print("파일이 업로드되지 않았습니다.")
else:
    xlsx_file_name = xlsx_file_names[0]  # 첫 번째 업로드된 파일 이름을 사용


# Excel 파일 읽기 (예: data.xlsx)
df_excel = pd.read_excel(xlsx_file_name, engine='openpyxl')


Saving PION_추출본_231004.xlsx to PION_추출본_231004.xlsx


In [97]:
# CSV에서 '단어'와 '빈도' 열 추출
csv_words = df_csv['단어'].tolist()
csv_frequencies = df_csv['빈도'].tolist()

# Excel에서 'Part Reference'와 'quantity' 열 추출
excel_part_references = filtered_excel['Part Reference'].tolist()
excel_quantities = filtered_excel['Quantity'].tolist()

# 'Part Reference'가 일치하면서 '단어'와 '빈도'가 같은 경우를 찾아 출력
matching_records = []
mismatching_records = []
zero_reccords = []


for excel_ref, excel_qty in zip(excel_part_references, excel_quantities):
    excel_ref_list = excel_ref.split(',')
    total_frequency = 0
    for ref in excel_ref_list:
        if ref in csv_words:
            csv_index = csv_words.index(ref)
            total_frequency += csv_frequencies[csv_index]
    if total_frequency == excel_qty:
        matching_records.append({
            'Part Reference': excel_ref,
            '빈도 합계': total_frequency,
            'quantity': excel_qty
        })
    elif total_frequency > excel_qty:  # 빈도 합계가 Quantity보다 큰 경우
        mismatching_records.append({
            'Part Reference': excel_ref,
            '빈도 합계': total_frequency,
            'quantity': excel_qty
        })

# 빈도가 Quantity보다 큰 경우만 출력
print("NC일 확률이 높은 레코드:")
for record in mismatching_records:
    print(record)


NC일 확률이 높은 레코드:
{'Part Reference': 'C1,C4', '빈도 합계': 11, 'quantity': 2}
{'Part Reference': 'C6,C9', '빈도 합계': 3, 'quantity': 2}
{'Part Reference': 'D1,D2', '빈도 합계': 8, 'quantity': 2}
{'Part Reference': 'R1,R9,R16,R23,R80,R85,R89', '빈도 합계': 8, 'quantity': 7}
{'Part Reference': 'R3,R7,R13,R72', '빈도 합계': 5, 'quantity': 4}
{'Part Reference': 'U2,U4', '빈도 합계': 4, 'quantity': 2}
{'Part Reference': 'X1', '빈도 합계': 2, 'quantity': 1}
